<a href="https://colab.research.google.com/github/mary270/Deep_Learning/blob/main/EMNIST_using_Transfer_learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Use the MNIST weights and train a new model for EMNIST using transfer learning with those previous MNIST weights.

In [ ]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils

# Load MNIST data
(X_train_mnist, y_train_mnist), (X_test_mnist, y_test_mnist) = mnist.load_data()

# Load EMNIST data
# Load EMNIST data from local file
# Replace <PATH_TO_EMNIST> with the path to the EMNIST dataset file
emnist_data = np.load('/content/emnist-letters-train-images-idx3-ubyte', allow_pickle=True)

X_train_emnist, y_train_emnist = emnist_data['X_train'], emnist_data['y_train']
X_test_emnist, y_test_emnist = emnist_data['X_test'], emnist_data['y_test']

# Preprocess data
X_train_mnist = X_train_mnist.reshape(X_train_mnist.shape[0], 28, 28, 1).astype('float32') / 255
X_test_mnist = X_test_mnist.reshape(X_test_mnist.shape[0], 28, 28, 1).astype('float32') / 255

X_train_emnist = X_train_emnist.reshape(X_train_emnist.shape[0], 28, 28, 1).astype('float32') / 255
X_test_emnist = X_test_emnist.reshape(X_test_emnist.shape[0], 28, 28, 1).astype('float32') / 255

y_train_mnist = np_utils.to_categorical(y_train_mnist)
y_test_mnist = np_utils.to_categorical(y_test_mnist)

y_train_emnist = np_utils.to_categorical(y_train_emnist)
y_test_emnist = np_utils.to_categorical(y_test_emnist)

# Load pre-trained MNIST model weights
mnist_model = keras.models.load_model('mnist_model.h5')

# Create a new model for EMNIST
emnist_model = Sequential()

# Copy the architecture of the MNIST model (except for the output layer)
for layer in mnist_model.layers[:-1]:
    emnist_model.add(layer)

# Add a new output layer for EMNIST
emnist_model.add(Dense(47, activation='softmax'))  # 47 classes in EMNIST

# Compile the model
emnist_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Transfer the weights from the pre-trained MNIST model to the new EMNIST model
for i in range(len(emnist_model.layers) - 1):
    emnist_model.layers[i].set_weights(mnist_model.layers[i].get_weights())

# Train the EMNIST model
emnist_model.fit(X_train_emnist, y_train_emnist, validation_data=(X_test_emnist, y_test_emnist), batch_size=128, epochs=10)

# Evaluate the EMNIST model
emnist_loss, emnist_acc = emnist_model.evaluate(X_test_emnist, y_test_emnist)
print('EMNIST Test Loss:', emnist_loss)
print('EMNIST Test Accuracy:', emnist_acc)

# Save the trained EMNIST model
emnist_model.save('emnist_model.h5')
